In [48]:
import dataclasses
import typing
import statistics

@dataclasses.dataclass
class MyType:
    a: int
    b: float
    
    @classmethod
    def from_number(cls, i: int):
        return cls(i, 1/(i+1))
    
@dataclasses.dataclass
class MyTypeTwo:
    sum: int
    prod: float
    
    @classmethod
    def from_mytype(cls, mt: MyType):
        return cls(sum = mt.a + mt.b, prod = mt.a * mt.b)

@dataclasses.dataclass
class MyCollectionTwo:
    objs: typing.List[MyType] = dataclasses.field(default_factory=list)
    
    @classmethod
    def from_mycollection(cls, mytypes: MyCollection):
        return MyCollectionTwo([MyTypeTwo.from_mytype(mt) for mt in mytypes])

class MyTypeAverage(MyType):
    @classmethod
    def from_mytypes(cls, mtypes: typing.Iterable[MyType]):
        return cls(
            a = statistics.mean([o.a for o in mtypes]),
            b = statistics.mean([o.b for o in mtypes]),
        )

In [51]:
import multiprocessing

@dataclasses.dataclass
class MyCollection:
    objs: typing.List[MyType] = dataclasses.field(default_factory=list)
    
    def __iter__(self) -> typing.Iterator[MyType]:
        return iter(self.objs)
    
    def __getitem__(self, ind: int) -> MyType:
        return self.objs[ind]
    
    @classmethod
    def from_ab_pairs(cls, elements: typing.Iterable):
        return cls([MyType(*el) for el in elements])
    
    def append(self, *args, **kwargs) -> None:
        return self.objs.append(*args, **kwargs)
    
    def append_mytype(self, *args, **kwargs) -> None:
        return self.objs.append(MyType(*args, **kwargs))
    
    def append_from_number(self, *args, **kwargs) -> None:
        return self.objs.append(MyType.from_number(*args, **kwargs))
    
    @classmethod
    def from_numbers(cls, numbers: typing.Iterable[int]):
        return cls([MyType.from_number(i) for i in numbers])
    
    def filter(self, keep_if: typing.Callable[[MyType], bool]):
        return self.__class__([o for o in self.objs if keep_if(o)])
    
    def average(self) -> MyTypeAverage:
        return MyTypeAverage(
            a = statistics.mean([o.a for o in self.objs]),
            b = statistics.mean([o.b for o in self.objs]),
        )
    
    def average_sfm(self) -> MyTypeAverage:
        return MyTypeAverage.from_mytypes(self.objs)
    
    def group_by_average(self, *args, **kwargs) -> typing.Dict[typing.Hashable, MyTypeAverage]:
        return {k:grp.average() for k,grp in self.group_by(*args, **kwargs).items()}

    def group_by_as_dict(self, key_func: typing.Callable[[MyType], typing.Hashable]) -> typing.Dict[str, MyCollection]:
        groups = dict()
        for el in self.objs:
            k = key_func(el)
            if k not in groups:
                groups[k] = list()
            groups[k].append(el)
        return {k:self.__class__(grp) for k,grp in groups.items()}
    
    def group_by(self, key_func: typing.Callable[[MyType], typing.Hashable]):
        groups = dict()
        for el in self.objs:
            k = key_func(el)
            if k not in groups:
                groups[k] = list()
            groups[k].append(el)
        return GroupedMyCollection({k:self.__class__(grp) for k,grp in groups.items()})
    
    def transform_to_two(self) -> MyCollectionTwo:
        return MyCollectionTwo.from_mycollection(self)
    
    def transform_parallelized(self) -> MyCollectionTwo:
        with multiprocessing.Pool() as p:
            results = p.map(MyTypeTwo.from_mytype, self)
        return MyCollectionTwo(results)

class GroupedMyCollection(typing.Dict[typing.Hashable, MyCollection]):
    def average(self) -> typing.Dict[typing.Hashable, MyTypeAverage]:
        return {k:grp.average() for k,grp in self.items()}
    
MyCollection([])
MyCollection()
mc = MyCollection([])
mc.append(MyType(1, 2.0))
mytypes = MyCollection([MyType(i, 1/(i+1)) for i in range(10)])
mytypes = MyCollection.from_numbers(range(10))
mytypes.append(MyType(1, 2.0))
mytypes.append_mytype(1, 2.0)
mytypes.append_from_number(1)
mytypes[0]
[mt for mt in mytypes]
mytypes.filter(lambda mt: mt.a > 4)
mytypes.average()
mytypes.average_sfm()
print(mytypes.group_by(lambda mt: int(mt.a) % 2 == 0))
print(mytypes.group_by(lambda mt: int(mt.a) % 2 == 0).average())
mytypes.group_by_average(lambda mt: int(mt.a) % 2 == 0)
mytypes.transform_to_two()
mytypes.transform_parallelized()

{True: MyCollection(objs=[MyType(a=0, b=1.0), MyType(a=2, b=0.3333333333333333), MyType(a=4, b=0.2), MyType(a=6, b=0.14285714285714285), MyType(a=8, b=0.1111111111111111)]), False: MyCollection(objs=[MyType(a=1, b=0.5), MyType(a=3, b=0.25), MyType(a=5, b=0.16666666666666666), MyType(a=7, b=0.125), MyType(a=9, b=0.1), MyType(a=1, b=2.0), MyType(a=1, b=2.0), MyType(a=1, b=0.5)])}
{True: MyTypeAverage(a=4, b=0.3574603174603175), False: MyTypeAverage(a=3.5, b=0.7052083333333333)}


MyCollectionTwo(objs=[MyTypeTwo(sum=1.0, prod=0.0), MyTypeTwo(sum=1.5, prod=0.5), MyTypeTwo(sum=2.3333333333333335, prod=0.6666666666666666), MyTypeTwo(sum=3.25, prod=0.75), MyTypeTwo(sum=4.2, prod=0.8), MyTypeTwo(sum=5.166666666666667, prod=0.8333333333333333), MyTypeTwo(sum=6.142857142857143, prod=0.8571428571428571), MyTypeTwo(sum=7.125, prod=0.875), MyTypeTwo(sum=8.11111111111111, prod=0.8888888888888888), MyTypeTwo(sum=9.1, prod=0.9), MyTypeTwo(sum=3.0, prod=2.0), MyTypeTwo(sum=3.0, prod=2.0), MyTypeTwo(sum=1.5, prod=0.5)])

In [37]:
class MyCollectionExtended(typing.List[MyType]):
    @classmethod
    def from_numbers(cls, numbers: typing.Iterable[int]):
        return cls(MyType.from_number(i) for i in numbers)
    
    @classmethod
    def from_ab_pairs(cls, elements: typing.Iterable):
        return cls(MyType(*el) for el in elements)
    
    def append_mytype(self, *args, **kwargs) -> None:
        return self.append(MyType(*args, **kwargs))
    
    def append_from_number(self, *args, **kwargs) -> None:
        return self.append(MyType.from_number(*args, **kwargs))

mytypes = MyCollectionExtended.from_numbers(range(10))
mytypes.from_numbers(range(5))
mytypes.append_from_number(1)